![title](logo.png)

# Today: Image recognition with MXNet
- Image pre-processing
- Convolutional Neural Networks
- Architecture aspects
- Hyperparameter considerations
- Optimization algorithms


![title](nn.jpeg)

![title](cnn.jpeg)

# Data pre-processing
- Scaling
- Normalization
- Labeling

### Inspect data

In [ ]:
raw = load(string("datatide4/petter_00002.jpg"))

In [ ]:
using DataFrames
using FileIO
using Images

In [ ]:
#A = float(rawview(channelview(temp))) # Use to go between float matrices and pictures
imageFolder = "./datatide4/"
filenames = map(x -> replace(x, ".jpg", ""), readdir(imageFolder))
labels = map(x -> split(x, "_")[1], filenames)
classes = unique(labels)
classDict = Dict(classes[i] => (i-1) for i=1:length(classes))
labelSize = 1
width = 75
heigth = 75

data = zeros(Float32, heigth, width, length(filenames))
label = zeros(Int64, length(filenames))

for i in 1:length(filenames)
    image = load(string("datatide4/", filenames[i], ".jpg"))
    image_resized = imresize(image, heigth, width)
    temp = convert(Array{Float32}, image_resized)
    data[ :, :, i] = temp 
    label[i] = classDict[labels[i]]
end

### Input data to CNN after pre-processing

In [ ]:
colorview(Gray,data[:,:,40])

### Create a MXNet data provider from the data

In [ ]:
using MXNet

In [ ]:
mxData = mx.Variable(:data)
mxLabel  = mx.Variable(:softmax_label)

batch_size = 5

train_provider = mx.ArrayDataProvider(:data => reshape(data, (heigth,width,1,length(filenames))),
    :softmax_label => label ,
    batch_size=batch_size, 
    shuffle=true)


# Create CNN architecture
- kernel
- filters
- pooling
- stride

![title](pooling.jpeg)

![title](stride.png)

In [ ]:
conv1 = @mx.chain mx.Convolution(mxData, kernel=(3,3), num_filter=200)  =>
                  mx.BatchNorm() =>
                  mx.Activation(act_type=:relu) =>
                  mx.Pooling(pool_type=:max, kernel=(3,3), stride=(2,2))

conv2 = @mx.chain mx.Convolution(conv1, kernel=(5,5), num_filter=200) =>
                  mx.BatchNorm() =>
                  mx.Activation(act_type=:relu) =>
                  mx.Pooling(pool_type=:max, kernel=(2,2), stride=(1,1))

conv3 = @mx.chain mx.Convolution(conv2, kernel=(3,3), num_filter=250) =>
                  mx.BatchNorm() =>
                  mx.Activation(act_type=:relu) =>
                  mx.Pooling(pool_type=:max, kernel=(2,2), stride=(2,2))

conv4 = @mx.chain mx.Convolution(conv3, kernel=(3,3), num_filter=250) =>
                  mx.BatchNorm() =>
                  mx.Activation(act_type=:relu) =>
                  mx.Pooling(pool_type=:max, kernel=(2,2), stride=(2,2))

fc1   = @mx.chain mx.Flatten(conv4) =>
                  mx.FullyConnected(num_hidden=400) =>
                  mx.BatchNorm() =>
                  mx.Activation(act_type=:relu) =>
                  mx.Dropout(p=0.3)

fc2   = @mx.chain mx.FullyConnected(fc1, num_hidden=600) =>
                  mx.BatchNorm() =>
                  mx.Activation(act_type=:relu) =>
                  mx.Dropout(p=0.3)

fc3   = @mx.chain mx.FullyConnected(fc2, num_hidden=400) =>
                  mx.BatchNorm() =>
                  mx.Activation(act_type=:relu) =>
                  mx.Dropout(p=0.3)

fc4   = mx.FullyConnected(fc3, num_hidden=length(classes)) 

mlp  = mx.SoftmaxOutput(fc4, name=:softmax)

In [ ]:
exec = mx.simple_bind(mlp, mx.cpu(), input=(75,75,1,36))
dbg_str = mx.debug_str(exec)

# Create model, define optimizer, run model

### Learning rate

![title](lr.png)

### Momentum

$v = \alpha v - \epsilon \bigtriangledown _w \Big(\frac{1}{m} \sum_{i=1}^{m}L(f(x^{i}; w), y^{i}) \Big)$ <br/>
$w \leftarrow w + v$

![title](gradient.png)

In [ ]:
const learning_rate = 0.1
const momentum = 0.90
const l2 = 0 # set to zero if dropout is used

# Define model and its hardware context
model = mx.FeedForward(mlp, context=[mx.gpu()])

# Define an optimizer to train the network
optimizer = mx.SGD(lr=0.001, momentum=0.8, weight_decay=0.00001) #(lr=0.01, weight_decay=0.00001)


# Some other built in optimizers

# mx.ADAM(lr = learning_rate, weight_decay=l2)

# mx.RMSProp(lr = learning_rate)




## Time to train!

In [ ]:
const epochs = 20

mx.fit(model,
    optimizer, 
    train_provider,
    eval_metric=mx.Accuracy(),
    n_epoch=epochs, 
    callbacks=[mx.speedometer(frequency=20),
    mx.do_checkpoint("/home/laketide/work/microsoft/checkpoints/LakeNet", frequency=epochs)]
    )

# Let's see some results!

In [ ]:
e_data = zeros(Float32, heigth, width, 1) 
e_label = zeros(Int64, 1)

image = load(string("/home/laketide/work/microsoft/eval/petter_00001.jpg"))
image_resized = imresize(image, heigth, width)
temp = convert(Array{Float32}, image_resized)
e_data[:,:,1] = temp 

eval_provider = mx.ArrayDataProvider(:data => reshape(e_data, (heigth,width,1,1)),
    batch_size=1);

In [ ]:
probs = mx.predict(model, eval_provider)
alex = probs[1]
carlos = probs[2]
diant = probs[3]
peter = probs[4]
petter = probs[5]
robert = probs[6]
println("Probabilities:\n
    Alejandro: $alex\n
    Carlos: $carlos\n
    Diantha: $diant\n
    Peter: $peter\n
    Petter: $petter\n
    Robert: $robert")
image

In [ ]:
#Download these images to see full presentation
download("http://lh6.ggpht.com/-5RFMcz2bzWI/VA9NuUkB_vI/AAAAAAAAAdM/_DqqYO0nbX4/im2_thumb%25255B1%25255D.png?imgmax=800", "gradient.png")
download("http://cs231n.github.io/assets/nn1/neural_net.jpeg", "nn.jpeg")
download("http://cs231n.github.io/assets/cnn/maxpool.jpeg", "pooling.jpeg")
download("http://cs231n.github.io/assets/cnn/cnn.jpeg", "cnn.jpeg")
download("https://adeshpande3.github.io/assets/Stride1.png", "stride.png")
download("https://media.licdn.com/media/AAEAAQAAAAAAAAi4AAAAJDQ0ODNhMmEwLTVmN2MtNDhiZC05MTQyLWY5NTU5ZDllM2YzNQ.png", "logo.png")
download("https://raw.githubusercontent.com/qingkaikong/blog/master/2017_05_More_on_applying_ANN/figures/figure_4.png", "lr.png")